# TensorFlow Lite model for deployment

In [1]:
import os
import numpy as np
import tensorflow as tf

In [2]:
# Load the model
model = tf.keras.models.load_model('../models/efficientnetb0_14_0.962.h5')

In [3]:
# Path of unseen images
eval_imgs_dir = '../datasets/kitchenware-dataset/eval-images'

# Get the images filepaths
custom_imgs = [eval_imgs_dir + '/' + img_path for img_path in os.listdir(eval_imgs_dir)]

# Get the cup image
cup_img = custom_imgs[0]

In [4]:
# Import tensorflow library to load image
from tensorflow.keras.preprocessing.image import load_img

In [15]:
# Load image
img = load_img(cup_img, target_size=(256, 256))

# Convert image to numpy array and add batch dimension
x = np.array(img, dtype='float32') # must be float32 for tflite model
X = np.expand_dims(x, axis=0)
X

array([[[[110.,  80.,  69.],
         [108.,  78.,  67.],
         [115.,  85.,  74.],
         ...,
         [ 94.,  70.,  60.],
         [ 94.,  70.,  60.],
         [ 94.,  70.,  60.]],

        [[114.,  85.,  71.],
         [115.,  86.,  72.],
         [116.,  87.,  73.],
         ...,
         [ 84.,  59.,  52.],
         [ 91.,  67.,  57.],
         [ 91.,  67.,  55.]],

        [[117.,  88.,  72.],
         [110.,  81.,  67.],
         [120.,  91.,  77.],
         ...,
         [ 94.,  69.,  62.],
         [ 95.,  71.,  61.],
         [ 93.,  69.,  57.]],

        ...,

        [[ 19.,   7.,   7.],
         [ 17.,   6.,   4.],
         [ 18.,   7.,   5.],
         ...,
         [ 68.,  46.,  33.],
         [ 68.,  46.,  35.],
         [ 70.,  48.,  37.]],

        [[ 19.,  10.,   5.],
         [ 17.,   8.,   3.],
         [ 16.,   7.,   2.],
         ...,
         [ 67.,  43.,  31.],
         [ 71.,  47.,  35.],
         [ 67.,  43.,  33.]],

        [[ 18.,   7.,   5.],
       

In [6]:
# Make model prediction on the cup image
preds = model.predict(X)[0]
preds

1/1 [==============================] - 4s 4s/step


array([9.8942554e-01, 7.5928824e-07, 1.0567816e-02, 5.2624791e-06,
       4.8197681e-08, 4.8940933e-07], dtype=float32)

In [7]:
# List of classes the model was trained to predict
classes = ['cup', 'fork', 'glass', 'knife', 'plate', 'spoon']

# Check the prediction scores
dict(zip(classes, preds))

{'cup': 0.98942554,
 'fork': 7.5928824e-07,
 'glass': 0.010567816,
 'knife': 5.262479e-06,
 'plate': 4.819768e-08,
 'spoon': 4.8940933e-07}

## Convert keras model to tf-lite

In [8]:
# Initialize tf-lite converter for keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Make model conversion
tflite_model = converter.convert()

# Save the model in tflite format
with open('../models/kitchenware-model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\awon\AppData\Local\Temp\tmpcgcg6kyc\assets


INFO:tensorflow:Assets written to: C:\Users\awon\AppData\Local\Temp\tmpcgcg6kyc\assets


In [9]:
# Check the saved models and their sizes
!ls -lh ../models/

total 35M
-rw-r--r-- 1 awon 197121 19M Dec 15 06:52 efficientnetb0_14_0.962.h5
-rw-r--r-- 1 awon 197121 17M Dec 16 15:14 kitchenware-model.tflite


## Make predictions with tflite model

In [10]:
# Import tensorflow lite module
import tensorflow.lite as tflite

In [11]:
# Instantiate model using Interpreter class
interpreter = tflite.Interpreter(model_path='../models/kitchenware-model.tflite')
# Load the weights from the model to memory
interpreter.allocate_tensors()

# Get the input index from interpreter
input_index = interpreter.get_input_details()[0]['index']
# Get the output index from interpreter
output_index = interpreter.get_output_details()[0]['index']

In [16]:
# Set the value of the input tensor using input_index to the image X
interpreter.set_tensor(input_index, X)
# Invoke the interpreter
interpreter.invoke()

# Get the value of the output tensor using output_index to make prediction
tflite_preds = interpreter.get_tensor(output_index)[0]
tflite_preds

array([9.8942566e-01, 7.5928978e-07, 1.0567832e-02, 5.2624650e-06,
       4.8197595e-08, 4.8940842e-07], dtype=float32)

In [17]:
# Check the prediction scores
dict(zip(classes, preds))

{'cup': 0.98942554,
 'fork': 7.5928824e-07,
 'glass': 0.010567816,
 'knife': 5.262479e-06,
 'plate': 4.819768e-08,
 'spoon': 4.8940933e-07}

## Removing TF dependencies

In [1]:
import tflite_runtime.interpreter as tflite

In [2]:
# Initalize interpreter
interpreter = tflite.Interpreter(model_path='../models/kitchenware-model.tflite')
# Allocate memory
interpreter.allocate_tensors()

# Get input index
input_index = interpreter.get_input_details()[0]['index']
# Get output index
output_index = interpreter.get_output_details()[0]['index']

In [3]:
import numpy as np
from io import BytesIO
from urllib import request
from PIL import Image
from PIL.Image import Resampling


def preprocess_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, resample=Resampling.NEAREST)
    x = np.array(img, dtype='float32')
    X = np.expand_dims(x, axis=0)
    return X

def image_from_url(url, target_size=(256, 256)):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return preprocess_image(img, target_size)

def image_from_path(path, target_size=(256, 256)):
    with Image.open(path) as img:
        return preprocess_image(img, target_size)

In [4]:
# From url
url = 'https://static-01.daraz.pk/p/1dec6d4a0810d17bd9f1d835f192d922.jpg'

# Create X matrix from image url
X_url = image_from_url(url, target_size=(256, 256))

In [5]:
import os

# Get the images filepaths
custom_imgs = ['../datasets/kitchenware-dataset/eval-images' + '/' + img_path for img_path in os.listdir('../datasets/kitchenware-dataset/eval-images')]
# Get the cup image
img_path = custom_imgs[0]

# Create X matrix from path url
X = image_from_path(img_path, target_size=(256, 256))

In [6]:
# Make prediction on cup image from path
interpreter.set_tensor(input_index, X)
interpreter.invoke()
tflite_runtime_preds = interpreter.get_tensor(output_index)[0]

# List of classes
classes = ['cup', 'fork', 'glass', 'knife', 'plate', 'spoon']

# Check the prediction scores
dict(zip(classes, tflite_runtime_preds))

{'cup': 0.98942566,
 'fork': 7.592926e-07,
 'glass': 0.010567902,
 'knife': 5.2625e-06,
 'plate': 4.8197872e-08,
 'spoon': 4.8941126e-07}

In [7]:
# Make prediction on cup image from url
interpreter.set_tensor(input_index, X_url)
interpreter.invoke()
tflite_runtime_preds = interpreter.get_tensor(output_index)[0]

# List of classes
classes = ['cup', 'fork', 'glass', 'knife', 'plate', 'spoon']

# Check the prediction scores
dict(zip(classes, tflite_runtime_preds))

{'cup': 1.0,
 'fork': 2.8106922e-10,
 'glass': 2.9442784e-09,
 'knife': 3.0271075e-10,
 'plate': 6.8399513e-12,
 'spoon': 8.5472615e-11}